In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import pickle

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/My Drive/ai_project/data/f10-7.pkl', 'rb') as f:
    df = pickle.load(f)

In [ ]:
n = 500
df.head()

,Date,value
44,19470305,306.10
45,19470306,306.00
46,19470307,321.40
47,19470308,345.60
48,19470309,335.75
49,19470310,325.90
50,19470311,303.30
51,19470312,305.30
52,19470313,359.50
53,19470314,267.20


In [ ]:
X = np.array([df['value'].values[i:i+n] for i in range(len(df)-n)])
y = np.array([df['value'].values[i+n] for i in range(len(df)-n)])

In [ ]:
len(X), len(y)

(25489, 25489)

In [ ]:
X.shape, y.shape

((25489, 500), (25489,))

In [ ]:
X = X.reshape((X.shape[0], X.shape[1], 1))

In [ ]:
X_test, y_test = X[-n:], y[-n:]

In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(np.array(X[:-n]), np.array(y[:-n]), test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((19991, 500, 1), (4998, 500, 1), (19991,), (4998,))

In [ ]:
def plot_result(his):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    ax1.plot(his.history['loss'])
    ax1.plot(his.history['val_loss'])
    ax1.set_title('Model Loss')
    ax1.set_ylabel('Loss')
    ax1.set_xlabel('Epoch')
    ax1.legend(['Train', 'Validation'], loc='upper right')

    ax2.plot(his.history['mae'])
    ax2.plot(his.history['val_mae'])
    ax2.set_title('Model MAE')
    ax2.set_ylabel('MAE')
    ax2.set_xlabel('Epoch')
    ax2.legend(['Train', 'Validation'], loc='upper right')

    plt.show()

In [ ]:
def make_pred(num, model):
    data_pred = np.array(df['value'].values[:-n])
    for i in range(num):
        temp = data_pred[-n:]
        pred = model.predict(temp.reshape(1, -1))
        data_pred = np.append(data_pred, pred)
    return data_pred[-num:]

In [ ]:
model1 = Sequential()
# model1.add(SimpleRNN(200, dropout=0.2, recurrent_dropout=0.2))
model1.add(SimpleRNN(200, return_sequences=True, input_shape=[None, 1]))
model1.add(SimpleRNN(100, return_sequences=True))
model1.add(SimpleRNN(20))
model1.add(Dense(1))

model1.compile(loss='mse', optimizer='adam', metrics=['mae'])

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, None, 200)         40400     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, None, 100)         30100     
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 20)                2420      
                                                                 
 dense (Dense)               (None, 1)                 21        
                                                                 
Total params: 72941 (284.93 KB)
Trainable params: 72941 (284.93 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history1 = model1.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    verbose=1,
                    validation_data=(X_val, y_val))

Epoch 1/10
625/625 [==============================] - 323s 516ms/step - loss: 13915.7305 - mae: 105.6965 - val_loss: 12804.0322 - val_mae: 100.2030
Epoch 2/10
625/625 [==============================] - 321s 514ms/step - loss: 11507.4365 - mae: 93.6285 - val_loss: 10581.6514 - val_mae: 88.4209
Epoch 3/10
625/625 [==============================] - 339s 542ms/step - loss: 9480.3740 - mae: 82.1157 - val_loss: 8714.6104 - val_mae: 77.1441
Epoch 4/10
625/625 [==============================] - 340s 544ms/step - loss: 7791.6104 - mae: 71.0375 - val_loss: 7170.5083 - val_mae: 66.3861
Epoch 5/10
625/625 [==============================] - 336s 538ms/step - loss: 6405.0679 - mae: 60.5682 - val_loss: 5909.1255 - val_mae: 56.2113
Epoch 6/10
625/625 [==============================] - 340s 544ms/step - loss: 5288.8618 - mae: 51.8538 - val_loss: 4908.3096 - val_mae: 49.4352
Epoch 7/10
625/625 [==============================] - 339s 543ms/step - loss: 4420.1733 - mae: 46.7896 - val_loss: 4142.3853 - val

In [ ]:
def plot_result(his):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    ax1.plot(his.history['loss'])
    ax1.plot(his.history['val_loss'])
    ax1.set_title('Model Loss')
    ax1.set_ylabel('Loss')
    ax1.set_xlabel('Epoch')
    ax1.legend(['Train', 'Validation'], loc='upper right')

    ax2.plot(his.history['mae'])
    ax2.plot(his.history['val_mae'])
    ax2.set_title('Model MAE')
    ax2.set_ylabel('MAE')
    ax2.set_xlabel('Epoch')
    ax2.legend(['Train', 'Validation'], loc='upper right')

    plt.show()

def make_pred(num, model):
    data_pred = np.array(df['value'].values[:-n])
    for i in range(num):
        temp = data_pred[-n:]
        pred = model.predict(temp.reshape(1, -1))
        data_pred = np.append(data_pred, pred)
    return data_pred[-num:]



In [ ]:
plot_result(history1)

In [ ]:
pred1 = make_pred(300, model1)

In [ ]:
plt.plot(pred1, label='pred')
plt.plot(y_test[:300], label='real')
plt.legend()

In [ ]:
n = 300
plt.plot(recent['value'].values[-n-100:-100], label='before', color='blue')
plt.plot(range(n, n+100), pred1, label='pred', color='orange')
plt.plot(range(n, n+100), y_test, label='real', color='green')
plt.legend()